<figure>
  <center><img style='height: 100%; width: 100%; object-fit: contain' src="../images/mars_rover.jpg" /></center>
  <center><figcaption>How do we optimize decision making in uncertain environments?</figcaption></center>
</figure>

### **Agents and Environments**

Finite Markov Decision Processes (MDPs) are fundamental processes in reinforcement learning (RL) and are an extension of [markov chains]({filename}../probability/markov_chains.ipynb), systems where the next state is only dependent on the current state. MDPs extend Markov chains by simply adding in actions and rewards, hence turning a stochastic process into a stochastic decision process. They are useful for studying optimization problems solved using dynamic programming, which is a method for solving problems with optimal substructure, that is, where the optimal solution to a problem can be found by combining optimal solutions to its subproblems. In this post, we'll go over the basics of MDPs and how they are used in RL.

MDPs involve both evaluative feedback, as with the k-armed bandits, but also an **associative** aspect, where **actions influence not just immediate rewards, but also subsequent situations (or states)**. This leads to delayed rewards and the need to trade off immediate and delayed rewards because the state the bot finds itself in can impact future rewards. In this sense, the RL agent is trying to determine how both their environment and actions map onto value. In k-armed bandits, the goal is to estimate the value $q_{*}(a)$ of each action $a$, while with MDPs, the goal is to estimate $q_{*}(s,a)$ of each action $a$ in each state $s$, or the value $v_{*}(s)$ of each state given optimal action selections. It's important to note that MDPs are a mathematically idealized form of the RL problem.

The agent-environment interface is the foundation of MDPs. The agent, or the learner and decision maker, interacts with the environment, which presents new situations and gives rise to rewards that the agent seeks to maximize over time with its actions. The agent and environment interact at discrete time steps $t=0,1,2,3,...$ and at each time step, the agent receives a representation of the environment's state, $S_t \in S$ and from that, selects an action, $A_t \in A(s)$. One time step later, the agent receives a reward $R_{t+1} \in R$ and finds itself in a new state $S_{t+1}$. The state space can be anything including stock returns, the score of a game, or the position of a robot. 
 A common diagram used to represent this, as well as RL in general, is the following:

<figure>
  <center><img style='height: 80%; width: 80%; object-fit: contain' src="../images/MDP.PNG" /></center>
  <center><figcaption>A Markov Decision Process involves an agent interacting with an environment. At discrete time steps, the agent takes an action and then recieves a reward and state in the next time step. The process repeats.</figcaption></center>
</figure>

In a finite MDP, the sets of states, actions, and rewards ($S, A, R$) all have a finite number of elements. In this case, the random variables $R_t$ and $S_t$ have well defined discrete probability distributions dependent only on the preceding state and action. **The probability distribution which completely characterizes the dynamics of the MDP is theoretically represented as**:

$$p(s',r|s,a) := Pr(S_t=s',R_t=r|S_{t-1}=s,A_{t-1}=a)$$

where $s,s' \in S, r \in R, a \in A(s)$. This is the probability of ending up in state $s'$ and receiving reward $r$ given that the agent was in state $s$ and took action $a$. The agent-environment interaction is modeled as a Markov process with state $S_t$ and state transition probability $p(s',r|s,a)$.

The state-transition probabilities can be computed as: 

$$p(s'|s,a) := Pr(S_t=s'|S_{t-1}=s,A_{t-1}=a) = \sum_{r \in R}p(s',r|s,a)$$

and the expected rewards for state-action pairs can be computed as:

$$r(s,a) := \mathbf{E}[R_{t}|S_{t-1}=s,A_{t-1}=a] = \sum_{r \in R}r\sum_{s' \in S}p(s',r|s,a)$$

The general rule is that whatever cannot be changed by the agent is considered part of its environment. However, the reward computation is always considered external to the agent because it defines the task facing the agent and is beyond its ability to change arbitrarily. The agent-environment boundary represents the limit of the agent's absolute control, not of its knowledge.

Any problem of goal-directed behavior can be reduced to 3 signals passed back and forth between the agent and the environment:

1. **Reward signal**: defines the agent's goals
2. **State signal**: indicates the basis on which the agent chooses its actions
3. **Action signal**: indicates what the agent chooses to do

### **Goals, Policies and Value Functions**

The **reward hypothesis** posited by Sutton states that "all we mean by goals and purposes can be well thought of as the maximization of the expected value of the cumulative sum of a received scalar signal (called reward)." Taken with the idea that AI is the study of creating agents which achieve goals, some argue that RL is essentially the path to solving AI. Correspondingly, the reward signal we give the system is our way of telling the machine what we want it to achieve, but not how.  

In this sense, when building RL agents, we seek to maximize the expected return, or the sum of rewards, denoted as $G_t$ (think of it like gain in the financial sense):

$$G_t = R_{t+1} + R_{t+2} + R_{t+3} + ... + R_{T}$$

where $T$ is the final time step. Sometimes, the agent-environment interaction breaks up into subsequences called **episodes**, which terminate after a finite number of steps in a special state called the **terminal state**. The next episode then starts independently of how the previous one ended. 

Additionally, we have a concept of *discounting*, where we want to give more weight to rewards that are far into the future (think of it as making the machine more long term oriented). Modifying our expression above with a discount factor, we have:

$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... = \sum_{k=0}^{\infty}\gamma^k R_{t+k+1}$$

where $\gamma \in [0,1]$ is the discount factor. This factor allows us to make the agent more or less myopic, or short-sighted. If $\gamma = 0$, the agent is completely myopic and only cares about the immediate reward. If $\gamma = 1$, the agent is completely far-sighted and cares about the long term reward. The discounting also makes the infinite sum finite via the geometric series formula!

RL algorithms are all about estimating value function! What are value functions? They are functions of states (or of state-action pairs) that estimate how good it is for the agent to be in a given state (or how good it is to perform a given action in a given state). Specifically the value of a state is given in terms of an expected reward received in the future if it follows a certain **policy, or ways of acting (more specifically, policies map states to probabilities of selecting each possible action)**. 

The **state-value function** for policy $\pi$, denoted as $v_{\pi}(s)$, is the expected return when starting in state $s$ and following policy $\pi$ thereafter. Mathematically, it is defined as:

$$v_{\pi}(s) := \mathbf{E}_{\pi}[G_t|S_t=s] = \mathbf{E}_{\pi}[\sum_{k=0}^{\infty}\gamma^k R_{t+k+1}|S_t=s]$$

where $\mathbf{E}_{\pi}$ denotes the expected value of a random variable given that the agent follows policy $\pi$. The value of the terminal state is always zero because there are no more rewards to be had.

We can also define the **action-value function** for policy $\pi$, denoted as $q_{\pi}(s,a)$, as the expected return starting from state $s$, taking action $a$, and then following policy $\pi$ thereafter. Mathematically, it is defined as:

$$q_{\pi}(s,a) := \mathbf{E}_{\pi}[G_t|S_t=s,A_t=a] = \mathbf{E}_{\pi}[\sum_{k=0}^{\infty}\gamma^k R_{t+k+1}|S_t=s,A_t=a]$$

The action-value function is useful because it tells us how good it is to take a certain action in a given state under a certain policy. **In practice one can estimate $v_{\pi}$ and $q_{\pi}$ by averaging the returns observed after visits to a state $s$ and state-action pair $(s,a)$, respectively (Monte Carlo)**.

A fundamental property of value functions is that they satisfy a certain recursive relationship. For any policy $\pi$ and any state $s$, the following consistency condition holds between the value of $s$ and the value of its possible successor states:

$$v_{\pi}(s) = \sum_{a \in A(s)}\pi(a|s)\sum_{s' \in S, r \in R}p(s',r|s,a)[r + \gamma v_{\pi}(s')]$$

where $\pi(a|s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$. This is called the **Bellman equation** for $v_{\pi}$. It expresses the fact that the value of a state under policy $\pi$ must equal the expected return for all possible actions that can be taken from that state, weighted by the probability of taking those actions.

### **Optimal Policies and Optimal Value Functions**

At the end of the day, RL is about finding the policies that maximize the accumulation of reward over the long run, i.e. that maximize the value. Optimal policies have value functions that are greater than the value functions of the other policies **for all states**. We denote the optimal state-value function as $v_{*}(s)$ and the optimal action-value function as $q_{*}(s,a)$. The optimal state-value function is defined as:

$$v_{*}(s) := \max_{\pi}v_{\pi}(s)$$

and the optimal action-value function is defined as:

$$q_{*}(s,a) := \max_{\pi}q_{\pi}(s,a)$$

The optimal value functions satisfy a recursive relationship similar to the Bellman equation and is known as the **Bellman optimality equation**:

$$v_{*}(s) = \max_{a \in A(s)}\sum_{s' \in S, r \in R}p(s',r|s,a)[r + \gamma v_{*}(s')]$$

where the max is taken over all possible actions. For finite MDPs, there is a unique solution which can be determinined if you have the transition probabilities and rewards associated with those transitions. But the problem with solving the Bellman optimality equation is that it is akin to an exhaustive search, which is computationally intractable for large MDPs. We typically need to know the dynamics of the environment, we need plenty of computational resources and we need the Markov property to hold. In practice, some combination of these is usually not met. As an example, backgammon has a state space of $10^{20}$ states and would take a modern computer thousands of years to solve.

### **Conclusion**

RL is about learning from interaction but specifically with the intent of maximizing long term accumulation of rewards, which might be discounted in a certain way. MDPs are a way of modeling this interaction and are a fundamental concept in RL. The basic framework of an MDP are the agent-environment interface, which entails actions (choices made by the agent), states (the agent's perception of the environment), and rewards (the basis for evaluations actions); policies, which are a stochastic rule by which the agent selects an action given a state; and most importantly a transition probability function, which defines the dynamics of the agent-environment system. Much of current RL theory is restricted to finite MDPs where you have a finite number of states, actions, and rewards. In addition, we learning that value functions assign to each state, or state-action pair, the expected return starting from that state, or state-action pair, and following a policy thereafter. A policy whose value functions are optimal is called an optimal policy. We can solve the Bellman optimality equation to find the optimal value functions, from which we can derive the optimal policy easily. However, this is computationally intractable for large MDPs. In the next post, we'll go over how to solve MDPs using dynamic programming!

### **Sources**

Sutton, R. S., Barto, A. G. (2018). Reinforcement learning: An introduction (2nd ed.). MIT Press Ltd. 

White, M., White, A. (n.d.). *Reinforcement Learning Specialization* [MOOC]. Coursera.

Markov Chains | Brilliant Math & Science Wiki. (n.d.). Brilliant.org. https://brilliant.org/wiki/markov-chains/#:~:text=A%20Markov%20chain%20is%20a

Wikipedia Contributors. (2019, May 20). Markov decision process. Wikipedia; Wikimedia Foundation. https://en.wikipedia.org/wiki/Markov_decision_process

Bowling, Michael, et al. Settling the Reward Hypothesis. 2023.